In [165]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [ ]:
dic = {
    "Nazare-Surf-Report":194,
    "Pipeline-Backdoor-Surf-Report":616,
    "Mundaka-Surf-Report": 169,
    "El-Fronton-Surf-Report":1879,
    "The-Box-Surf-Report":2873,
    "Puerto-Escondido-Surf-Report":340,
    "Itacoatiara-Surf-Report":1273,
    "Padang-Padang-Surf-Report":1121,
    "Kirra-Surf-Report":528
    "Mosca-Point-Surf-Report":1875
    "Tauro-Surf-Report":1878
       
}

In [187]:
def login():
    driver = webdriver.Chrome("./chromedriver.exe")
    driver.get("https://magicseaweed.com/account/login/?dest=%2F")
    sleep(2)
    driver.find_element_by_css_selector("#msw-js-user-login-tab > form > div:nth-child(2) > input").send_keys("lukesmith194@hotmail.com")
    driver.find_element_by_css_selector("#msw-js-user-login-tab > form > div:nth-child(3) > input").send_keys("Versus.123")
    sleep(1)
    driver.find_element_by_css_selector("#msw-js-login").click()
    sleep(2)

    return driver

In [188]:
ranges = [("2020-01-01","2020-01-31"),("2020-02-01","2020-02-29"),("2020-03-01","2020-03-31"),("2020-04-01","2020-04-30"),("2020-05-01","2020-05-31"),("2020-06-01","2020-06-30"),("2020-07-01","2020-07-31"),("2020-08-01","2020-08-31"),("2020-09-01","2020-09-30"),("2020-10-01","2020-10-31"),("2020-11-01","2020-11-30"),("2020-12-01","2020-12-31")]

### Sacar fila por fila datos

In [189]:
driver = login()

In [190]:
def get_arrow_value(fila):
    col = fila.find_element_by_class_name("text-center.msw-js-tooltip.background-gray-lighter").get_attribute("data-original-title")
    direction, angle = col.split(" - ")
    
    angle = int(angle[:-1])
    
    return direction, angle

In [191]:
def get_wind_value(fila):
    try:
        col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-success").get_attribute("data-original-title")
    except:
        try:
            col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-warning").get_attribute("data-original-title")
        except:
            try:
                col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-danger").get_attribute("data-original-title")
            except:
                return "null", "null", np.nan
        
    
    text, angle = col.split(" - ")

    angle = int(angle[:-1])
    _, text = text.split(", ")

    text, direction = text.split()

    return text, direction, angle

In [192]:
def get_hora(fila):
    elems = fila.text.split("\n")
    elems = elems[0].split() + elems[1:]
    
    arrow = get_arrow_value(fila)
    wind = get_wind_value(fila)
    
    elems = elems + list(arrow) + list(wind)
    
    return elems

In [193]:
def get_dia(tabla):
    filas = tabla.find_elements_by_tag_name("tr")
    dia = filas[0].text.split()[1]
    
    # 6am 12pm 6pm son filas 4, 6, 8
    filas = filas[3:8:2]
    
    dias = [[dia] + get_hora(f) for f in filas]
    
    return dias

In [194]:
def get_range_dates(d, driver):
    # d is a tuple of 2 elements (start_date, end_date)
    # access webpage
        
    driver.get(f"https://magicseaweed.com/Tauro-Surf-Report/1878/Historic/?start={d[0]}&end={d[1]}")
    sleep(2)
    # extract 31 days' data
    table = driver.find_element_by_css_selector("#msw-js-fc > div.table-responsive-xs > table")

    tablitas = table.find_elements_by_tag_name("tbody")
    tablitas = tablitas[::3]

    mega_tabla = []

    for t in tablitas:
        dia = get_dia(t)
        mega_tabla.extend(dia)
        
    return mega_tabla

In [195]:
tabla_anio = []

for r in ranges:
    print(f"Rango {r}...")
    driver.get(f"https://magicseaweed.com/Tauro-Surf-Report/1878/Historic/?start={r[0]}&end={r[1]}")
    
    tabla_mes = get_range_dates(r, driver)
    
    tabla_anio.extend(tabla_mes)

Rango ('2020-01-01', '2020-01-31')...
Rango ('2020-02-01', '2020-02-29')...
Rango ('2020-03-01', '2020-03-31')...
Rango ('2020-04-01', '2020-04-30')...
Rango ('2020-05-01', '2020-05-31')...
Rango ('2020-06-01', '2020-06-30')...
Rango ('2020-07-01', '2020-07-31')...
Rango ('2020-08-01', '2020-08-31')...
Rango ('2020-09-01', '2020-09-30')...
Rango ('2020-10-01', '2020-10-31')...
Rango ('2020-11-01', '2020-11-30')...
Rango ('2020-12-01', '2020-12-31')...


In [196]:
df_tauro = pd.DataFrame(tabla_anio, columns=[
    "date", "time", "range", "height", "period", "wind_mph", "wind_kmh", "mph", "temp", "prob",
    "swell_dir", "swell_angle", "wind", "wind_dir", "wind_angle"#"x1","x2","x3","x4"
])

### NAZARE

In [30]:
df.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,7-11ft,5ft,11s,3,4,mph,6°c,100%,NW,306,Offshore,ESE,115
1,01/01,12pm,7-11ft,5ft,10s,4,6,mph,13°c,100%,NW,310,Offshore,SE,144
2,01/01,6pm,6-10ft,4.5ft,10s,1,1,mph,8°c,100%,NW,309,Offshore,S,171
3,02/01,6am,4-7ft,2.5ft,10s,5,6,mph,6°c,100%,NW,322,Offshore,SE,129
4,02/01,12pm,5-8ft,3ft,12s,6,9,mph,11°c,100%,W,276,Offshore,SSE,156


In [27]:
df.to_csv("firstdf.csv", index=False)

### Pipeline

In [61]:
df2.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,7-11ft,7.5ft,16s,17,20,mph,23°c,100%,NW,312,Offshore,ESE,108
1,01/01,12pm,6-10ft,7ft,15s,12,13,mph,24°c,100%,NW,312,Cross/Offshore,E,82
2,01/01,6pm,6-10ft,7ft,15s,9,10,mph,24°c,100%,NW,314,Onshore,NE,35
3,02/01,6am,5-8ft,6ft,14s,7,8,mph,23°c,100%,NW,313,Onshore,NNE,32
4,02/01,12pm,5-8ft,5.5ft,13s,10,11,mph,24°c,100%,NW,314,Cross/Onshore,NE,46


In [62]:
df2.to_csv("pipedf.csv", index=False)

### MUNDAKA

In [163]:
df_mu.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,3-4ft,4ft,12s,6,6,mph,10°c,100%,WNW,304,Offshore,SW,220
1,01/01,12pm,3-4ft,4.5ft,11s,6,6,mph,11°c,100%,NW,305,Offshore,SSW,194
2,01/01,6pm,3-4ft,4.5ft,11s,6,6,mph,12°c,100%,NW,307,Onshore,E,91
3,02/01,6am,2-3ft,3ft,10s,7,8,mph,10°c,100%,NW,311,Offshore,SSE,149
4,02/01,12pm,2-3ft,2.5ft,10s,8,9,mph,11°c,100%,NW,312,Offshore,S,183


In [164]:
df_mu.to_csv("mundaka_correct_df.csv", index=False)

### FRONTON

In [88]:
df4.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,1-2ft,1.5ft,12s,8,11,mph,12°c,100%,NW,321,Offshore,SSE,163
1,01/01,12pm,1-2ft,1.3ft,11s,14,14,mph,16°c,100%,NW,323,Offshore,SE,137
2,01/01,6pm,1-2ft,1.4ft,7s,8,9,mph,14°c,100%,NNE,33,Offshore,SE,136
3,02/01,6am,1ft,1ft,7s,9,12,mph,11°c,100%,NE,35,Offshore,SSE,151
4,02/01,12pm,1ft,0.9ft,12s,13,13,mph,15°c,100%,WNW,301,Offshore,SE,126


In [89]:
df4.to_csv("fronton.csv", index=False)

### The Box

In [100]:
df5.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,5-8ft,7.5ft,12s,5,5,mph,15°c,100%,SW,228,Offshore,ENE,57
1,01/01,12pm,5-8ft,7.5ft,12s,10,11,mph,19°c,100%,SW,228,Onshore,SW,228
2,01/01,6pm,5-8ft,7.5ft,12s,14,18,mph,19°c,100%,SW,230,Onshore,SSW,211
3,02/01,6am,7-11ft,9.5ft,15s,11,15,mph,18°c,100%,SW,236,Cross/Onshore,S,189
4,02/01,12pm,7-11ft,10ft,15s,17,17,mph,19°c,100%,WSW,236,Onshore,SW,222


In [101]:
df5.to_csv("box.csv", index=False)

### Puerto Escondido

In [112]:
df6.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,5-7ft,2.5ft,18s,4,5,mph,25°c,100%,SW,216,Cross/Offshore,NW,319
1,01/01,12pm,4-7ft,2.5ft,18s,7,7,mph,27°c,100%,SW,216,Onshore,SW,214
2,01/01,6pm,4-6ft,2.5ft,18s,10,13,mph,27°c,100%,SW,215,Cross/Onshore,W,277
3,02/01,6am,4-6ft,2.5ft,16s,5,6,mph,25°c,100%,SW,214,Offshore,NW,326
4,02/01,12pm,4-6ft,2.5ft,16s,12,14,mph,28°c,100%,SSW,213,Onshore,WSW,253


In [113]:
df6.to_csv("puerto_escondido.csv", index=False)

### Itacoatiara

In [124]:
df7.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,1ft,3ft,8s,7,9,mph,24°c,100%,E,99,Offshore,NNE,20
1,01/01,12pm,1ft,3ft,7s,10,11,mph,28°c,100%,E,98,Cross-shore,E,98
2,01/01,6pm,1ft,3ft,7s,13,16,mph,28°c,100%,E,97,Cross/Offshore,E,84
3,02/01,6am,1ft,3ft,7s,6,7,mph,25°c,100%,E,97,Cross/Onshore,WSW,254
4,02/01,12pm,1ft,2.5ft,7s,11,12,mph,25°c,100%,E,99,Onshore,S,187


In [125]:
df7.to_csv("itacoatiara.csv", index=False)

### Padang-padang

In [136]:
df8.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,Plano,7ft,14s,6,6,mph,28°c,100%,SSW,207,Onshore,WNW,284
1,01/01,12pm,Plano,7ft,15s,14,14,mph,28°c,100%,SSW,206,Cross/Offshore,NW,323
2,01/01,6pm,Plano,7ft,14s,7,8,mph,27°c,100%,SSW,207,Cross/Offshore,NW,316
3,02/01,6am,Plano,6.5ft,14s,7,6,mph,27°c,100%,SSW,207,Offshore,NNE,19
4,02/01,12pm,Plano,6ft,13s,9,10,mph,29°c,100%,SSW,208,Onshore,WSW,252


In [137]:
df8.to_csv("padang-padang.csv", index=False)

### Kirra

In [149]:
df9.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,Plano,5ft,12s,9,9,mph,24°c,100%,E,81,Onshore,NNE,15
1,01/01,12pm,Plano,5ft,12s,12,13,mph,26°c,100%,E,81,Onshore,NNE,26
2,01/01,6pm,Plano,5ft,12s,13,16,mph,25°c,100%,ENE,79,Onshore,NNE,29
3,02/01,6am,Plano,4.5ft,12s,8,9,mph,25°c,100%,E,80,Onshore,NNE,17
4,02/01,12pm,Plano,4.5ft,12s,13,13,mph,25°c,100%,E,80,Onshore,NE,37


In [150]:
df9.to_csv("kirra.csv", index=False)

### Mosca

In [176]:
df_mosca

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,1ft,1.4ft,7s,8,11,mph,12°c,100%,NNE,31,Cross/Onshore,SSE,163
1,01/01,12pm,1-2ft,2ft,4s,14,14,mph,16°c,100%,ESE,121,Onshore,SE,137
2,01/01,6pm,1-2ft,2.5ft,5s,8,9,mph,14°c,100%,ESE,122,Onshore,SE,136
3,02/01,6am,1-2ft,2ft,5s,9,12,mph,11°c,100%,ESE,113,Onshore,SSE,151
4,02/01,12pm,1-2ft,2.5ft,5s,13,13,mph,15°c,100%,ESE,111,Onshore,SE,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,30/12,12pm,2-3ft,9ft,15s,8,9,mph,18°c,N/A,N,359,Onshore,NE,52
1094,30/12,6pm,2-3ft,8.5ft,15s,4,4,mph,18°c,N/A,N,359,Onshore,ENE,67
1095,31/12,6am,2-4ft,9.5ft,15s,6,6,mph,16°c,N/A,N,1,Cross/Onshore,N,10
1096,31/12,12pm,2-3ft,9.5ft,15s,9,10,mph,19°c,N/A,N,2,Onshore,NE,43


In [177]:
df_mosca.to_csv("mosca.csv", index=False)

#### Tauro

In [197]:
df_tauro

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,1ft,0.8ft,9s,8,11,mph,12°c,100%,W,279,Onshore,SSE,163
1,01/01,12pm,0-1ft,0.6ft,8s,14,14,mph,16°c,100%,W,279,Cross-shore,SE,137
2,01/01,6pm,0-1ft,0.6ft,10s,8,9,mph,14°c,100%,WNW,284,Cross-shore,SE,136
3,02/01,6am,0-1ft,0.5ft,12s,9,12,mph,11°c,100%,WNW,298,Cross/Onshore,SSE,151
4,02/01,12pm,1ft,0.7ft,12s,13,13,mph,15°c,100%,WNW,293,Cross/Offshore,SE,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,30/12,12pm,1ft,0.6ft,13s,4,4,mph,18°c,N/A,SSW,203,Cross/Onshore,SSE,154
1094,30/12,6pm,1ft,0.7ft,13s,2,2,mph,18°c,N/A,SSW,203,Onshore,SW,228
1095,31/12,6am,1-2ft,0.9ft,12s,3,5,mph,17°c,N/A,SSW,202,Offshore,N,349
1096,31/12,12pm,1-2ft,1ft,12s,3,3,mph,19°c,N/A,SSW,203,Onshore,SSW,207


In [198]:
df_tauro.to_csv("tauro.csv", index=False)

### UPSCALING

In [151]:
"""def get_range_dates2(d, driver, dic):
    # d is a tuple of 2 elements (start_date, end_date)
    # access webpage
    for spot,code in dic.items():   
        driver.get(f"https://magicseaweed.com/{spot}/{code}/Historic/?start={d[0]}&end={d[1]}")
        sleep(2)
        # extract 31 days' data
        table = driver.find_element_by_css_selector("#msw-js-fc > div.table-responsive-xs > table")

        tablitas = table.find_elements_by_tag_name("tbody")
        tablitas = tablitas[::3]

        mega_tabla = []

        for t in tablitas:
            dia = get_dia(t)
            mega_tabla.extend(dia)
        
    return mega_tabla"""

In [ ]:
"""tabla_anio = []

for r in ranges:
    print(f"Rango {r}...")
    for spot,code in dic.items()
    driver.get(f"https://magicseaweed.com/{spot}/{code}/Historic/?start={r[0]}&end={r[1]}")
    
    tabla_mes = get_range_dates(r, driver)
    
    tabla_anio.extend(tabla_mes)"""